In [1]:
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from PIL import Image # Needed for handling images

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# 1. Data Preparation and Augmentation
# =====================================

# Define data transformations for training, validation, and testing
# Training data augmentation is crucial for improving model generalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),  # Randomly crop and resize to 224x224
        transforms.RandomHorizontalFlip(),  # Randomly flip horizontally
        transforms.ToTensor(),             # Convert to PyTorch tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize with ImageNet statistics
    ]),
    'test': transforms.Compose([ # Added test transforms.  Important to have consistent preprocessing.
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [5]:
class SoilDataset(Dataset):
    """
    A custom dataset class for loading soil images and labels from a separate file.
    """
    def __init__(self, data_dir, label_file, transform=None, phase='train', classes=None,
                class_to_idx=None):
        """
        Args:
            data_dir (str): Path to the directory containing the images.
            label_file (str): Path to the CSV file containing the labels.
            transform (callable, optional): Optional transform to be applied on a sample.
            phase (str):  'train', 'val', or 'test' to indicate the subset to load.
        """
        self.data_dir = data_dir
        self.label_file = label_file
        self.transform = transform
        self.phase = phase
        
        # Filter the dataframe based on the phase.  Assume label_file has a 'split' column.

        if phase == 'train':
            self.labels_df = pd.read_csv(label_file)
            self.image_names = self.labels_df['image_id'].tolist() # get the image file names
            self.labels = self.labels_df['soil_type'].tolist() # get the labels
            self.classes = sorted(list(set(self.labels)))  # Get unique class names
            self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)} # create a mapping
        else:
            self.image_names = [f for f in os.listdir(f"{data_dir}/{phase}") if os.path.isfile(os.path.join(f"{data_dir}/{phase}", f))]
            self.labels = ['Alluvial soil'] * len(self.image_names) # dummy labels
            self.classes = classes
            self.class_to_idx = class_to_idx
    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(f"{self.data_dir}/{self.phase}", img_name) # full path.
        image = Image.open(img_path).convert('RGB')  # Load the image

        label = self.labels[idx]
        label_idx = self.class_to_idx[label] # convert label name to index.

        if self.transform:
            image = self.transform(image)
        if self.phase == "test":
            label_idx = img_name
            

        return image, label_idx



data_dir = '/kaggle/input/soil-classification/soil_classification-2025/'  # Replace with the path to your image data directory
label_file = '/kaggle/input/soil-classification/soil_classification-2025/train_labels.csv'  # Replace with the path to your CSV label file
image_datasets = {
    'train': SoilDataset(data_dir, label_file, transform=data_transforms['train'], phase='train')}

image_datasets['test']=SoilDataset(data_dir, None, transform=data_transforms['train'], phase='test',
                                  classes=image_datasets['train'].classes,
                                  class_to_idx=image_datasets['train'].class_to_idx,)



In [6]:
# Create data loaders
#  -  These handle batching, shuffling, and parallel data loading
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=8, shuffle=True, num_workers=4),
    'test': DataLoader(image_datasets['test'], batch_size=8, shuffle=False, num_workers=4) # Create test dataloader
}

# Get the class names.  This is important for mapping predictions to soil types.
class_names = image_datasets['train'].classes
num_classes = len(class_names) # number of classes

print(f"Number of classes: {num_classes}")
print(f"Classes: {class_names}")

Number of classes: 4
Classes: ['Alluvial soil', 'Black Soil', 'Clay soil', 'Red soil']


In [7]:
# 7. Save the Model (Optional)
# ============================
def save_model(model, path='soil_classification_model.pth'):
    """Saves the trained model to a specified path.

    Args:
        model (torch.nn.Module): The trained model.
        path (str, optional): The path to save the model.
            Defaults to 'soil_classification_model.pth'.
    """
    torch.save(model.state_dict(), path)
    print(f'Model saved to {path}')

# Save the model



In [9]:
# 2. Model Definition ( with Fine-tuning)
# ==============================================
def create_efficient_net_model(num_classes, fine_tune=True):
    """
    Loads a pre-trained _efficient_net_ model and modifies the final fully connected layer
    for the specific number of classes in the soil classification task.  Optionally
    freezes the earlier layers.

    Args:
        num_classes (int): The number of soil classes.
        fine_tune (bool, optional): If True, fine-tunes the entire model.
            If False, freezes the base layers and only trains the classifier.
            Defaults to True.

    Returns:
        torch.nn.Module: The modified _efficient_net_ model.
    """
    model_ft = models.efficientnet_b0(weights='IMAGENET1K_V1') #(weights=models.ResNet50_Weights.IMAGENET1K_V1) # Use the pretrained weights

    # Get the number of input features for the final fully connected layer
    in_features = model_ft.classifier[1].in_features

    # Replace the final fully connected layer with a new one
    # that has the number of output features equal to the number of classes.
    model_ft.classifier[1] = nn.Linear(in_features, num_classes)

    if not fine_tune:
        # Freeze all the parameters in the network
        for param in model_ft.parameters():
            param.requires_grad = False
    model_ft.classifier[1].requires_grad = True

    return model_ft

In [10]:
def create_efficient_net_b1_model(num_classes, fine_tune=True):
    """
    Loads a pre-trained model and modifies the final fully connected layer.

    Args:
        num_classes (int): The number of output classes.
        fine_tune (bool): If True, all layers are trainable. If False, base layers are frozen.

    Returns:
        torch.nn.Module: The modified model.
    """
    model_ft = models.efficientnet_b2(weights='IMAGENET1K_V1')

    if not fine_tune:
        # Freeze all parameters in the feature extractor (base layers)
        for param in model_ft.parameters():
            param.requires_grad = False
        # Unfreeze the final classification layer if it was frozen by the above loop
        model_ft.classifier[1].requires_grad = True # Ensure the new FC layer is trainable

    # Get the number of input features for the final fully connected layer
    in_features = model_ft.classifier[1].in_features
    
    # Replace the final fully connected layer with a new one for `num_classes`
    model_ft.classifier[1] = nn.Linear(in_features, num_classes)

    return model_ft

In [11]:
# Create the model
efficient_net_model_ft = create_efficient_net_model(num_classes, fine_tune=True) # Set fine_tune to True to train the whole network
efficient_net_model_ft = efficient_net_model_ft.to(device)  # Move the model to the GPU if available

efficient_net_model_base = create_efficient_net_model(num_classes, fine_tune=True) # Set fine_tune to True to train the whole network
efficient_net_model_base = efficient_net_model_base.to(device)  # Move the model to the GPU if available

v1_model_ft = create_efficient_net_b1_model(num_classes, fine_tune=True) # Set fine_tune to True to train the whole network
v1_model_ft = v1_model_ft.to(device)  # Move the model to the GPU if available

v1_model_base = create_efficient_net_b1_model(num_classes, fine_tune=False) # Set fine_tune to True to train the whole network
v1_model_base = v1_model_base.to(device)  # Move the model to the GPU if available


# 3. Loss Function and Optimizer
# ==============================
# Define the loss function (CrossEntropyLoss is suitable for multi-class classification)
train_labels = torch.tensor([image_datasets["train"].class_to_idx[t] for t in image_datasets["train"].labels])
class_counts = torch.bincount(train_labels, minlength=num_classes).float()
total_samples = class_counts.sum()
weights_calculated = total_samples / (num_classes * class_counts)
print(f"\nCalculated Class Counts: {class_counts.tolist()}")
print(f"Calculated Class Weights: {weights_calculated.tolist()}")

#weights = 1/torch.tensor(counts_list, dtype=torch.float)
criterion_weighted = nn.CrossEntropyLoss(weight=weights_calculated.to(device))
criterion = nn.CrossEntropyLoss()

# Define the optimizer.
#  -  If fine_tune is False, only the parameters of the new fully connected layer will be updated.
#  -  If fine_tune is True, all parameters will be updated.
optimizer_efficient_ft = optim.AdamW(efficient_net_model_ft.parameters(), lr=0.001)
optimizer_efficient_base = optim.AdamW(efficient_net_model_base.parameters(), lr=0.0001)
optimizer_v1_ft = optim.AdamW(v1_model_ft.parameters(), lr=0.001)
optimizer_v1_base = optim.AdamW(v1_model_base.parameters(), lr=0.0001)

# Define a learning rate scheduler to decrease the learning rate over time
# This can help the model converge better.
exp_lr_scheduler_eff = lr_scheduler.StepLR(optimizer_efficient_ft, step_size=7, gamma=0.1)
exp_lr_scheduler_base = lr_scheduler.StepLR(optimizer_efficient_base, step_size=7, gamma=0.1)
exp_lr_scheduler_v1 = lr_scheduler.StepLR(optimizer_v1_ft, step_size=7, gamma=0.1)
exp_lr_scheduler_v1_base = lr_scheduler.StepLR(optimizer_v1_base, step_size=7, gamma=0.1)



Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 192MB/s]
Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 235MB/s]



Calculated Class Counts: [528.0, 231.0, 199.0, 264.0]
Calculated Class Weights: [0.5785984992980957, 1.322510838508606, 1.5351759195327759, 1.1571969985961914]


In [12]:
# 4. Model Training and Validation
# ==================================
def train_model(model, criterion, optimizer, scheduler, num_epochs=50):
    """
    Trains the model for a specified number of epochs, performing validation
    at the end of each epoch.

    Args:
        model (torch.nn.Module): The model to train.
        criterion (torch.nn.Module): The loss function.
        optimizer (torch.optim.Optimizer): The optimizer.
        scheduler (torch.optim.lr_scheduler._LRScheduler): The learning rate scheduler.
        num_epochs (int, optional): The number of epochs to train for. Defaults to 25.

    Returns:
        torch.nn.Module: The trained model.
    """
    best_acc = 0.0  # Initialize the best validation accuracy
    # Store the loss and accuracy for each epoch
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data in batches
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device) # Move the inputs to the device (GPU or CPU)
                labels = labels.to(device) # Move the labels to the device

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs) # Get the model's output
                    _, preds = torch.max(outputs, 1)  # Get the predicted class labels
                    loss = criterion(outputs, labels) # Calculate the loss

                    if phase == 'train':
                        loss.backward()        # Backpropagate the loss
                        optimizer.step()       # Update the model parameters

                # Statistics
                running_loss += loss.item() * inputs.size(0) # Accumulate the loss
                running_corrects += torch.sum(preds == labels.data)  # Accumulate the number of correct predictions

            # Calculate the loss and accuracy for this epoch
            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it's the best one seen so far
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict() # save the weights
                print(f'Best val Acc: {best_acc:.4f}')

        if phase == 'train': # Step the scheduler *after* the epoch
            scheduler.step()

    print(f'Finished Training. Best val Acc: {best_acc:.4f}')
    model.load_state_dict(best_model_wts) # load the weights of the best model
    return model, history



# Train the model
num_epochs = 35 # You can adjust the number of epochs

# try:
#     efficient_net_model_ft.load_state_dict(torch.load('soil_classification_model_efficient_net.pth', map_location=device))
#     efficient_net_model_base.load_state_dict(torch.load('soil_classification_model_efficient_net_base.pth', map_location=device))
#     resnet_model_ft.load_state_dict(torch.load('soil_classification_model_resnet.pth', map_location=device))
#     resnet_model_base.load_state_dict(torch.load('soil_classification_model_resnet_base.pth', map_location=device))
# except:
#     pass
efficient_net_model_ft, history = train_model(efficient_net_model_ft, criterion_weighted, optimizer_efficient_ft, exp_lr_scheduler_eff, num_epochs=num_epochs)
v1_model_ft, history = train_model(v1_model_ft, criterion_weighted, optimizer_v1_ft, exp_lr_scheduler_v1, num_epochs=num_epochs)
efficient_net_model_base, history = train_model(efficient_net_model_base, criterion_weighted, optimizer_efficient_base, exp_lr_scheduler_base, num_epochs=num_epochs)
v1_model_base, history = train_model(v1_model_base, criterion_weighted, optimizer_v1_base, exp_lr_scheduler_v1_base, num_epochs=num_epochs)


Epoch 0/34
----------
train Loss: 0.6952 Acc: 0.7406
Best val Acc: 0.7406
Epoch 1/34
----------
train Loss: 0.5859 Acc: 0.7856
Best val Acc: 0.7856
Epoch 2/34
----------
train Loss: 0.4869 Acc: 0.8093
Best val Acc: 0.8093
Epoch 3/34
----------
train Loss: 0.4183 Acc: 0.8429
Best val Acc: 0.8429
Epoch 4/34
----------
train Loss: 0.4171 Acc: 0.8470
Best val Acc: 0.8470
Epoch 5/34
----------
train Loss: 0.4365 Acc: 0.8404
Epoch 6/34
----------
train Loss: 0.3767 Acc: 0.8560
Best val Acc: 0.8560
Epoch 7/34
----------
train Loss: 0.2140 Acc: 0.9075
Best val Acc: 0.9075
Epoch 8/34
----------
train Loss: 0.2171 Acc: 0.9083
Best val Acc: 0.9083
Epoch 9/34
----------
train Loss: 0.1922 Acc: 0.9239
Best val Acc: 0.9239
Epoch 10/34
----------
train Loss: 0.1903 Acc: 0.9247
Best val Acc: 0.9247
Epoch 11/34
----------
train Loss: 0.1658 Acc: 0.9313
Best val Acc: 0.9313
Epoch 12/34
----------
train Loss: 0.1369 Acc: 0.9525
Best val Acc: 0.9525
Epoch 13/34
----------
train Loss: 0.1702 Acc: 0.9354
Ep

In [15]:
save_model(efficient_net_model_ft, path='soil_classification_model_efficient_net.pth')
save_model(efficient_net_model_base, path='soil_classification_model_efficient_net_base.pth')
save_model(v1_model_ft, path='soil_classification_model_v1.pth')
save_model(v1_model_base, path='soil_classification_model_v1_base.pth')

Model saved to soil_classification_model_efficient_net.pth
Model saved to soil_classification_model_efficient_net_base.pth
Model saved to soil_classification_model_v1.pth
Model saved to soil_classification_model_v1_base.pth


In [16]:
# 5. Model Evaluation on Test Set
# ================================
def evaluate_model(model1, model2, model3, model4, dataloader):
    """Evaluates the trained model on the test dataset.

    Args:
        model (torch.nn.Module): The trained model.
        dataloader (torch.utils.data.DataLoader): The DataLoader for the test set.
    """
    model1.eval()
    model2.eval()
    model3.eval()# Set the model to evaluation mode
    running_loss = 0.0
    running_corrects = 0
    all_preds = []
    image_ids = []

    # Iterate over the test data
    with torch.no_grad():  # Disable gradient calculation for evaluation
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            #labels = labels.to(device)

            outputs1 = model1(inputs)
            outputs2 = model2(inputs)
            outputs3 = model3(inputs)
            outputs4 = model4(inputs)
            _, preds1 = torch.max(outputs1, 1)
            _, preds2 = torch.max(outputs2, 1)
            _, preds3 = torch.max(outputs3, 1)
            _, preds4 = torch.max(outputs4, 1)
            stacked_predictions = torch.stack([preds1.squeeze(), preds2.squeeze(), preds3.squeeze(), preds4.squeeze()])
            # Find the mode along the first dimension (the models dimension)
            preds, _ = torch.mode(stacked_predictions, dim=0)
            image_ids.extend(labels)
            all_preds.extend(preds.cpu().numpy())
            #print(len(all_preds))# Store predictions and labels for later analysis

    return all_preds, image_ids  # Return the predictions and labels

# Evaluate the model on the test set
test_preds_test, test_image_ids = evaluate_model(efficient_net_model_ft, efficient_net_model_base, v1_model_ft,v1_model_base, dataloaders['test'])
test_preds_train, train_image_ids = evaluate_model(efficient_net_model_ft, efficient_net_model_base, v1_model_ft, v1_model_base, dataloaders['train'])

In [17]:
def find_keys_for_values(my_dict, value_list):
    """
    Finds keys in a dictionary corresponding to a list of values, 
    preserving the order of values.

    Args:
        my_dict (dict): The dictionary to search.
        value_list (list): A list of values to find keys for.

    Returns:
        list: A list of keys corresponding to the values in value_list, 
              in the same order. If a value is not found, it's skipped.
    """

    key_list = []
    for value in value_list:
        for key, val in my_dict.items():
            if val == value:
                key_list.append(key)
                break  # Move to the next value after finding a key
    return key_list
    

test_image_labels = find_keys_for_values(image_datasets["test"].class_to_idx,  test_preds_test) 
submission_df = pd.DataFrame({"image_id": test_image_ids, "soil_type": test_image_labels})
submission_df.head()

,image_id,soil_type
0,img_0f035b97.jpg,Clay soil
1,img_f13af256.jpg,Black Soil
2,img_15b41dbc.jpg,Black Soil
3,img_cfb4fc7a.jpg,Black Soil
4,img_683111fb.jpg,Black Soil


In [18]:
submission_df.to_csv("submission.csv")
submission_df.head()


,image_id,soil_type
0,img_0f035b97.jpg,Clay soil
1,img_f13af256.jpg,Black Soil
2,img_15b41dbc.jpg,Black Soil
3,img_cfb4fc7a.jpg,Black Soil
4,img_683111fb.jpg,Black Soil


In [ ]:
test_preds_test
